## Perform a GO enrichment analysis with GOATOOLS

In [24]:
import pandas as pd  # use data frame
import collections as cx  # use Counter
from goatools.obo_parser import GODag  # parse the .obo file (with GO ontology)
from goatools.anno.genetogo_reader import Gene2GoReader  # parse NCBI gene2go file
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS  # perform enrichment
from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj  # ploting

To do a GOEA, we need :
- a list of gene as background
- a list of gene to study
- a list of GO term
- a table of correspondance between gene ID and GO term

And, because GOATOOLS use NCBI Gene ID :
- a table of correspondance between NCBI Gene ID and Ensembl Gene ID

### Preparation of our gene background and gene list

In [2]:
similarity_table = pd.read_csv('../../res/exhaustive.similarity.txt', sep='\t')
ensembl2ncbi_df = pd.read_csv('../../data/gene2ensembl.gz', sep='\t', compression='gzip')

In [5]:
ensembl2ncbi_dict = dict(zip(ensembl2ncbi_df['Ensembl_gene_identifier'], ensembl2ncbi_df['GeneID']))

In [10]:
background = similarity_table['gene_id'].tolist()
nb_gene_available = len(background)
background = [ensembl2ncbi_dict[ensembl_id] for ensembl_id in background if ensembl_id in ensembl2ncbi_dict]
nb_gene_found = len(background)
print(f"{nb_gene_found}/{nb_gene_available} used as background")

19202/19204 used as background


In [11]:
gene_list = similarity_table['gene_id'][similarity_table['BP_similarity_longest'] < 0.9].tolist()
nb_gene_available = len(gene_list)
gene_list = [ensembl2ncbi_dict[ensembl_id] for ensembl_id in gene_list if ensembl_id in ensembl2ncbi_dict]
nb_gene_found = len(gene_list)
print(f"{nb_gene_found}/{nb_gene_available} used as gene_list")

523/524 used as gene_list


### Preparation of GOATOOLS

In [12]:
obo_dag = GODag("../../data/go-basic.obo")  # no obsolete

../../data/go-basic.obo: fmt(1.2) rel(2024-01-17) 45,869 Terms


In [16]:
gene2go = Gene2GoReader('../../data/gene2go', taxids=[9606])

HMS:0:01:10.668344 346,071 annotations, 20,759 genes, 18,733 GOs, 1 taxids READ: ../../data/gene2go 


In [17]:
ns2assoc = gene2go.get_ns2assc()

In [18]:
goea_obj = GOEnrichmentStudyNS(
    pop = background,
    ns2assoc = ns2assoc,
    godag = obo_dag,
    propagate_counts = False,
    alpha = 0.05,
    methods = ['fdr_bh']
)


Load BP Ontology Enrichment Analysis ...
 88% 16,856 of 19,202 population items found in association

Load CC Ontology Enrichment Analysis ...
 93% 17,936 of 19,202 population items found in association

Load MF Ontology Enrichment Analysis ...
 91% 17,411 of 19,202 population items found in association


### Run Analysis

In [19]:
goea_result = goea_obj.run_study(gene_list)


Runing BP Ontology Analysis: current study set of 523 IDs.
 99%    517 of    523 study items found in association
100%    523 of    523 study items found in population(19202)
Calculating 12,165 uncorrected p-values using fisher_scipy_stats
  12,165 terms are associated with 16,856 of 19,202 population items
   2,406 terms are associated with    517 of    523 study items
  METHOD fdr_bh:
       4 GO terms found significant (< 0.05=alpha) (  4 enriched +   0 purified): statsmodels fdr_bh
      51 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Runing CC Ontology Analysis: current study set of 523 IDs.
 98%    515 of    523 study items found in association
100%    523 of    523 study items found in population(19202)
Calculating 1,799 uncorrected p-values using fisher_scipy_stats
   1,799 terms are associated with 17,936 of 19,202 population items
     519 terms are associated with    515 of    523 study items
 

In [20]:
goea_result_sig = [r for r in goea_result if r.p_fdr_bh < 0.05]

### Ploting and writing results

In [22]:
print('{N} of {M:,} results were significant'.format(
    N=len(goea_result_sig),
    M=len(goea_result)))

32 of 18,557 results were significant


In [23]:
print('Significant results: {E} enriched, {P} purified'.format(
    E=sum(1 for r in goea_result_sig if r.enrichment=='e'),
    P=sum(1 for r in goea_result_sig if r.enrichment=='p')))


Significant results: 30 enriched, 2 purified


In [27]:
ctr = cx.Counter([r.NS for r in goea_result_sig])
print('Significant results[{TOTAL}] = {BP} BP + {MF} MF + {CC} CC'.format(
    TOTAL=len(goea_result_sig),
    BP=ctr['BP'],  # biological_process
    MF=ctr['MF'],  # molecular_function
    CC=ctr['CC'])) # cellular_component

Significant results[32] = 4 BP + 7 MF + 21 CC


In [29]:
goea_obj.wr_tsv("goea_results.tsv", goea_result_sig)
plot_results("sig_{NS}.png", goea_result_sig)

     32 items WROTE: goea_results.tsv
    4 usr  34 GOs  WROTE: sig_BP.png
   21 usr  46 GOs  WROTE: sig_CC.png
    7 usr  20 GOs  WROTE: sig_MF.png
